In [1]:
import os
import xarray as xr
import pygrib
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split

2023-06-14 21:35:05.350166: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-14 21:35:05.388108: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-14 21:35:05.389284: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-14 21:35:05.931902: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Function to read GRIB file and extract data
def read_grib_file(grib_file):
    grbs = pygrib.open(grib_file)
    grb = grbs.select()[0]

    # Extract the data from the GRIB message
    data = grb.values
    latitudes, longitudes = grb.latlons()

    grbs.close()

    return data, latitudes, longitudes

In [3]:
def preprocess_data(data):
    # Normalize data (optional, based on your requirements)
    normalized_data = (data - np.mean(data)) / np.std(data)

    return normalized_data


In [4]:
# Function to build and train the CNN model
def build_cnn_model(input_shape):
    model = Sequential()

    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(361 * 720, activation='relu'))  # Output layer with shape (361, 720)
    model.add(Reshape((361, 720)))

    model.compile(loss='mean_squared_error', optimizer='adam')

    return model


In [5]:
def load_data(folder_path):
    subfolders = ['000', '012']
    X = []
    Y = []

    for subfolder in subfolders:
        subfolder_path = os.path.join(folder_path, subfolder)
        files = os.listdir(subfolder_path)

        for file in files:
            grib_file_12h = os.path.join(subfolder_path, file)
            grib_file_0h = os.path.join(folder_path, '000', file)

            if os.path.isfile(grib_file_12h) and os.path.isfile(grib_file_0h):
                # Read and preprocess the data
                data_12h, latitudes, longitudes = read_grib_file(grib_file_12h)
                data_0h, _, _ = read_grib_file(grib_file_0h)

                # Preprocess the data
                X.append(preprocess_data(data_12h))
                Y.append(preprocess_data(data_0h))

    return np.array(X), np.array(Y)


In [6]:
data_folder = '../download'  # Folder path containing the GRIB files


In [7]:
# Load the data
X, Y = load_data(data_folder)


In [8]:
print(len(Y))
print(len(X))

640
640


In [9]:
# Reshape the data to match the input shape of the model
X = X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)

In [10]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)



In [11]:
# Print the shape of X_train for debugging
print(X_train.shape)  
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(512, 361, 720, 1)
(512, 361, 720)
(128, 361, 720, 1)
(128, 361, 720)


In [12]:
# Build the CNN model
input_shape = X_train.shape[1:]

model = build_cnn_model(input_shape)



2023-06-14 21:36:07.900940: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 2137948047360 exceeds 10% of free system memory.
2023-06-14 21:36:07.901027: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at stateless_random_ops_v2.cc:64 : RESOURCE_EXHAUSTED: OOM when allocating tensor with shape[2056352,259920] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu


ResourceExhaustedError: {{function_node __wrapped__StatelessRandomUniformV2_device_/job:localhost/replica:0/task:0/device:CPU:0}} OOM when allocating tensor with shape[2056352,259920] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu [Op:StatelessRandomUniformV2]

In [16]:
print(input_shape)

(361, 720, 1)


In [19]:
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=4)



Epoch 1/10


ValueError: in user code:

    File "/home/data/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "/home/data/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/data/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "/home/data/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1051, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/home/data/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1109, in compute_loss
        return self.compiled_loss(
    File "/home/data/.local/lib/python3.10/site-packages/keras/engine/compile_utils.py", line 265, in __call__
        loss_value = loss_obj(y_t, y_p, samp

In [ ]:
# Make predictions (replace X_test with your test data)
#predictions = model.predict(X_test)
#predictions



In [ ]:
# Perform any further processing or analysis with the predictions
# Hereafter, you can do whatever you want.

/home/data/detectives/data-detectives
